<a href="https://colab.research.google.com/github/emirunlu26/future-context-ST/blob/main/code/asr_mt_test_on_whisper_nllb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:

threshold = 750 # number of segment informations to read

In [ ]:
import numpy as np
import librosa

duration_class_by_upper_bound = {"1":5,"2":10,"3":15,"4":20}
duration_class_by_size = {"1":0,"2":0,"3":0,"4":0}
duration_class_by_percentage = {"1":0,"2":0,"3":0,"4":0}

highest_duration = 0

for index in range(threshold):
  audio_name = "audio_" + str(index) + ".wav"
  audio_path = "/content/drive/MyDrive/ColabNotebooks/audio_segments_test_set/" + audio_name
  audio, sample_rate = librosa.load(audio_path,sr=None)
  duration = len(audio) / sample_rate

  if duration > highest_duration:
    highest_duration = duration

  for duration_class in duration_class_by_upper_bound:
    if duration < duration_class_by_upper_bound[duration_class]:
      duration_class_by_size[duration_class] += 1
      break
print("\n\n")
print("Highest duration is:", highest_duration)
print(duration_class_by_size)

for duration_class in duration_class_by_percentage:
  duration_class_by_percentage[duration_class] = round(duration_class_by_size[duration_class] / threshold,3) * 100

print(duration_class_by_percentage)






In [ ]:
import numpy as np
import librosa
import soundfile as sf

start_index = 500
threshold = 750

# STORE THE SEGMENT INFORMATIONS IN segment_infos LIST
segment_infos = list()

with open("/content/drive/MyDrive/ColabNotebooks/segments.lst","r") as segment_file:
  segment_lines = segment_file.readlines()

  print(f"Total number of segment infos is {len(segment_lines)}")
  segment_lines = segment_lines[start_index:]
  for line in segment_lines:
    segment_info = line[0:len(line)-1].split()
    segment_infos.append(segment_info)

    if len(segment_infos) == threshold:
      break

  segment_infos = np.array(segment_infos)

  print(f"Length of segment_infos is {len(segment_infos)}")

  '''
  last_unique_audio = None
  unique_audioset = list()
  for segment_info in segment_infos:
    if last_unique_audio == segment_info[0]:
      continue
    unique_audioset.append(segment_info[0])
    last_unique_audio = segment_info[0]
    print(segment_info[0])
  '''

# GET THE AUDIO DATA FOR THE AUDIO SEGMENTS
  audio_segments = list()

  for index, segment in enumerate(segment_infos):
    audio_path = "/content/drive/MyDrive/ColabNotebooks/audios_test_set/" + segment[0] + ".m4a"
    audio_data, sample_rate = librosa.load(audio_path)
    start_sec = float(segment[1])
    end_sec = float(segment[2])
    start_sample = int(start_sec * sample_rate)
    end_sample = int(end_sec * sample_rate)
    audio_segment = audio_data[start_sample:end_sample]

    # Define output path
    output_path = f"/content/drive/MyDrive/ColabNotebooks/audio_segments_test_set/audio_{index}.wav"

    # Save the segment
    sf.write(output_path, audio_segment, sample_rate)
    print(f"Audio number {index+1} written ...")

    audio_segments.append(audio_segment)

**en.20101216.19.4-271.m4a**

GET THE TRANSCRIPTIONS BELONGING TO THE FIRST 500 AUDIO SEGMENTS

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/segments.en","r") as transcr_file:
  ref_transcriptions = list()
  for transcription in transcr_file:
    transcription = transcription[:len(transcription)-1]
    ref_transcriptions.append(transcription)
    if len(ref_transcriptions) == threshold:
      break

GET THE GERMAN TRANSLATIONS OF THE FIRST 500 TRANSCRIPTIONS

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/segments.de","r") as translate_file:
  ref_translations = list()
  for translation in translate_file:
    translation = translation[:len(translation)-1]
    ref_translations.append(translation)
    if len(ref_translations) == threshold:
      break
  ref_translations = [ref_translations]

**GET MODEL BY USING PIPELINE**

In [ ]:
!pip install transformers
!pip install -U openai-whisper
!pip install --upgrade soundfile

In [ ]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

whisper_transcriptions = map(whisper, audio_segments)
whisper_transcriptions = list(map(lambda x: x["text"], whisper_transcriptions))

**GET MODEL BY CALLING whisper.load_model()**

In [ ]:
import whisper
import librosa
import numpy as np

model = whisper.load_model("turbo").to("cuda")
whisper_transcriptions = list()

for index in range(threshold):
  audio_path = "/content/drive/MyDrive/ColabNotebooks/audio_segments_new/audio_" + str(index) + ".wav"
  audio, sample_rate = librosa.load(audio_path,sr=None)
  audio = whisper.pad_or_trim(np.array(audio).astype("float32"))
  mel = whisper.log_mel_spectrogram(audio,n_mels=model.dims.n_mels).to(model.device)

  options = whisper.DecodingOptions(language="en")
  result = whisper.decode(model, mel, options)
  whisper_transcriptions.append(result.text)

**LOAD THE PREDICTIONS GENERATED BY WHISPER IN SIMULEVAL**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Whisper-Wait-k/predicted_transcriptions.txt","r") as whisper_simuleval_predictions:
  simuleval_transcriptions = whisper_simuleval_predictions.readlines()
  simuleval_transcriptions = list(map(lambda x: x[:len(x)-1], simuleval_transcriptions))

**EVALUATE SIMULEVAL WHISPER RESULTS**

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(ref_transcriptions, simuleval_transcriptions):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())


EVALUATE WHISPER BY USING **jiwer**

In [ ]:
!pip install jiwer

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)

# CALCULATE MATCH ERROR RATE
mer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  mer_scores.append(mer(ref, pred))

mer_scores = np.array(mer_scores)

# CALCULATE WORD INFORMATION LOST
wil_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  wil_scores.append(wil(ref, pred))

wil_scores = np.array(wil_scores)

# CALCULATE WORD INFORMATION PRESERVED

wip_scores = np.ones(len(wil_scores)) - wil_scores

# CALCULATE CHARACTER ERROR RATE
cer_scores = list()

for ref, pred in zip(ref_transcriptions, whisper_transcriptions):
  cer_scores.append(cer(ref, pred))

cer_scores = np.array(cer_scores)

In [ ]:
print(wer_scores.mean())
print(wil_scores.mean())
print(wip_scores.mean())

STORE THE MEAN OF EACH EVALUATION METRIC IN A DATAFRAME

In [ ]:
import pandas as pd

whisper_eval_results = {"WER": wer_scores.mean(), "MER": mer_scores.mean(), "WIL": wil_scores.mean(), "WIP": wip_scores.mean(), "CER": cer_scores.mean()}
#whisper_eval_results = {"WER": wer_scores.mean()}

asr_eval_results_df = pd.DataFrame(whisper_eval_results, index=["Whisper Turbo"])
asr_eval_results_df.index.name = "ASR Model"
asr_eval_results_df.title = "ASR Evaluation Results on First 30 Audio Segments"

asr_eval_results_df.to_csv("/content/drive/MyDrive/ColabNotebooks/asr_eval_results.csv")

GET THE TRANSLATION PREDICTIONS GENERATED BY **NLLB** (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name,src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# DEFINE FUNCTION THAT APPLIES TRANSLATION

def from_eng_to_ger(eng_text):
  input_tokens = tokenizer(eng_text, return_tensors="pt").to("cuda")
  output_tokens = model.generate(
    **input_tokens, forced_bos_token_id=tokenizer.convert_tokens_to_ids("deu_Latn"), max_length=50)
  ger_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
  return ger_text

# TRANSLATE SEGMENTED TEXTUAL DATA BY USING NLLB MODEL

source_texts = ref_transcriptions
nllb_translations = list(map(from_eng_to_ger, source_texts))

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name,src_lang="eng_Latn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

text = ref_transcriptions[22]
source_text = text
source_text_encoding = tokenizer(source_text,return_tensors = "pt").to("cuda")

output_tokens = model.generate(
            **source_text_encoding,
            decoder_input_ids = None,
            forced_bos_token_id = tokenizer.convert_tokens_to_ids("deu_Latn"),
            max_new_tokens = 50,
        )

output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

print("Input id:", source_text_encoding["input_ids"])
print()
print("Output text:", output_text)
print()
print("Output tokens:", output_tokens)

**GET THE TRANSLATION PREDICTIONS GENERATED BY MARIAN NMT (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)**

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-de"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to("cuda")

def from_eng_to_ger_marian(eng_text):
  input_tokens = tokenizer(eng_text, return_tensors="pt", padding=True).to("cuda")
  output_tokens = model.generate(**input_tokens)
  ger_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
  return ger_text

source_texts = ref_transcriptions
marian_translations = list(map(from_eng_to_ger_marian, source_texts))

**EVALUATION OF SIMULEVAL TRANSLATIONS**

In [ ]:
path = "/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/nllb_output.txt"
#path = "/content/drive/MyDrive/ColabNotebooks/results/nllb_online_ref_transcriptions (simuleval)/nllb_k_3_output_2/output.txt"
with open(path,"r") as simuleval_file:
  simuleval_translations = simuleval_file.readlines()
  simuleval_translations = list(map(lambda x: x[:len(x)-1], simuleval_translations))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/nllb_k_5_output_2.txt","r") as simuleval_file:
  simuleval_translations_2 = simuleval_file.readlines()
  simuleval_translations_2 = list(map(lambda x: x[:len(x)-1], simuleval_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()
upper_limit = 0.9
count = 0

for i in range(500):
  wer_score = wer(simuleval_translations[i], simuleval_translations_2[i])
  if wer_score > upper_limit:
    count +=1
    print("Index:", i)
    print("WER:", wer_score)
    print("Translation 1:", simuleval_translations[i])
    print("Translation 2:", simuleval_translations_2[i] + "\n")
print("\nCount:", count)

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations, ref_translations)

print(nllb_bleu.score)
print(nllb_bleu)

**EVALUATION OF SIMULEVAL TRANSLATIONS - THE END**

EVALUATE NLLB BY USING **sacreBLEU** (INPUT: TRANSCRIPTIONS PROVIDED BY EUROPARL)

In [ ]:
!pip install sacrebleu

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)
print(nllb_bleu)

In [ ]:
import sacrebleu
import pandas as pd

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)

nllb_bleu_df = pd.DataFrame({"sacreBLEU(on 500 segments)": nllb_bleu.score}, index=["nllb-200-distilled-600M"])
nllb_bleu_df.index.name = "MT Model"

nllb_bleu_df.to_csv("/content/drive/MyDrive/ColabNotebooks/nllb_bleu.csv")


In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/nllb_offline_output.txt", "w", encoding="utf-8") as target:
  for translation in nllb_translations:
    target.write(translation + "\n")

**GET THE TRANSLATION PREDICTIONS GENERATED BY NLLB (INPUT: TRANSCRIPTIONS GENERATED BY WHISPER)**

In [ ]:
nllb_translations = list(map(from_eng_to_ger, whisper_transcriptions))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/results/nllb_offline_hypo_transcriptions/output.txt", "w", encoding="utf-8") as target:
  for translation in nllb_translations:
    target.write(translation + "\n")

**EVALUATE NLLB BY USING sacreBLEU (INPUT: TRANSCRIPTIONS GENERATED BY WHISPER)**

In [ ]:
import sacrebleu
import pandas as pd

nllb_bleu = sacrebleu.corpus_bleu(nllb_translations, ref_translations)

nllb_bleu_df = pd.DataFrame({"sacreBLEU(on 500 segments)": nllb_bleu.score}, index=["nllb-200-distilled-600M"])
nllb_bleu_df.index.name = "MT Model"

nllb_bleu_df.to_csv("/content/drive/MyDrive/ColabNotebooks/evaluations/nllb_offline_hypo_transcriptions/nllb_bleu_hypothesis_input.csv")


**COMPARE OUTPUTS OF DIFFERENT CASCADED SETUPS (MODELS NOT INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/translation_simuleval_2.txt","r") as simuleval_file:
  simuleval_translations = simuleval_file.readlines()
  simuleval_translations = list(map(lambda x: x[:len(x)-1], simuleval_translations))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/NLLB-Wait-k/translation_simuleval_4.txt","r") as simuleval_file:
  simuleval_translations_2 = simuleval_file.readlines()
  simuleval_translations_2 = list(map(lambda x: x[:len(x)-1], simuleval_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(simuleval_translations, simuleval_translations_2):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations, ref_translations)

print(nllb_bleu.score)

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(simuleval_translations_2, ref_translations)

print(nllb_bleu.score)

**CALCULATE BLEU SCORE OF OUTPUTS FROM REAL CASCADED SETUPS (MODELS INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_2_results.txt","r") as cascaded_output_file:
  cascaded_translations = cascaded_output_file.readlines()
  cascaded_translations = list(map(lambda x: x[:len(x)-1], cascaded_translations))

In [ ]:
import sacrebleu

nllb_bleu = sacrebleu.corpus_bleu(cascaded_translations, ref_translations)

print(nllb_bleu.score)

**COMPARE OUTPUTS FROM REAL CASCADED SETUPS WITH EACH OTHER (MODELS INTERCONNECTED)**

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_3_results.txt","r") as cascaded_output_file:
  cascaded_translations_1 = cascaded_output_file.readlines()
  cascaded_translations_1 = list(map(lambda x: x[:len(x)-1], cascaded_translations_1))

In [ ]:
with open("/content/drive/MyDrive/ColabNotebooks/SimulEval/Cascaded-Wait-k/cascaded_setup_4_results.txt","r") as cascaded_output_file_2:
  cascaded_translations_2 = cascaded_output_file_2.readlines()
  cascaded_translations_2 = list(map(lambda x: x[:len(x)-1], cascaded_translations_2))

In [ ]:
import numpy as np
from jiwer import wer, mer, wil, wip, cer

# CALCULATE WORD ERROR RATE
wer_scores = list()

for ref, pred in zip(cascaded_translations_1, cascaded_translations_2):
  wer_scores.append(wer(ref, pred))

wer_scores = np.array(wer_scores)
print(wer_scores.mean())